####  Here we orgnize the data in new files and plot the results

In [1]:
import numpy as np
#from scipy.io import loadmat
import seaborn as sns
from scipy import signal
from scipy.optimize import curve_fit
import cmath
import matplotlib.pyplot as plt
plt.style.use('seaborn-darkgrid')
#import hdf5storage
import json
import os

In [4]:

pars = ['center','sigma','amplitude_real','amplitude_imag']
parameters = {}

for i,file_name in enumerate(os.listdir('new_results2')):
    ext = os.path.splitext(file_name)[1] #file extension
    if ext == '.txt':
        with open(f'new_results2/{file_name}', 'r') as file:
            print(file)
            fit_data = json.load(file)
            data_ = {k : fit_data[k] for k in fit_data if fit_data[k]['params'] != [] }

            for k in data_:
                cen = [data_[k]["params"][4*(m-1)] for m in range(1,19)]
                order = sorted(range(18), key=lambda l: cen[l])
                keys = sorted(range(18), key=lambda l: order[l])
                parameters[k] = {}
                for n in range(18):
                    parameters[k][f'lz{keys[n] + 1}']  = {}
                    for i in range(4):
                        parameters[k][f'lz{keys[n] + 1}'][pars[i]] =  data_[k]['params'][(4*n)+ i]
            #        for i in range(4):

file_name = 'new_results2/parameters.txt'
with open(file_name, 'w') as file:
    file.truncate(0)
    file.write(json.dumps(parameters))
        

<_io.TextIOWrapper name='new_results2/101.txt' mode='r' encoding='cp1252'>
<_io.TextIOWrapper name='new_results2/105.txt' mode='r' encoding='cp1252'>
<_io.TextIOWrapper name='new_results2/13.txt' mode='r' encoding='cp1252'>
<_io.TextIOWrapper name='new_results2/17.txt' mode='r' encoding='cp1252'>
<_io.TextIOWrapper name='new_results2/21.txt' mode='r' encoding='cp1252'>
<_io.TextIOWrapper name='new_results2/25.txt' mode='r' encoding='cp1252'>
<_io.TextIOWrapper name='new_results2/29.txt' mode='r' encoding='cp1252'>
<_io.TextIOWrapper name='new_results2/33.txt' mode='r' encoding='cp1252'>
<_io.TextIOWrapper name='new_results2/37.txt' mode='r' encoding='cp1252'>
<_io.TextIOWrapper name='new_results2/41.txt' mode='r' encoding='cp1252'>
<_io.TextIOWrapper name='new_results2/45.txt' mode='r' encoding='cp1252'>
<_io.TextIOWrapper name='new_results2/49.txt' mode='r' encoding='cp1252'>
<_io.TextIOWrapper name='new_results2/5.txt' mode='r' encoding='cp1252'>
<_io.TextIOWrapper name='new_results2

KeyError: 'params'

## Frequency analysis
### Fitted data

In [5]:
amp_real = {n: np.zeros([118,118]) for n in range(1,19)}
amp_imag = {n: np.zeros([118,118]) for n in range(1,19)}

for i,k in enumerate(parameters):
    X = int(k.split(',')[0].split('[')[1])
    Y = int(k.split(',')[1].split(']')[0])
    for n in range(1,19):
        amp_real[n][X,Y] = parameters[k][f'lz{n}']['amplitude_real']
        amp_imag[n][X,Y] = parameters[k][f'lz{n}']['amplitude_imag']


In [6]:
print(f"Frequency: {parameters[k][f'lz3']['center']} Hz")

Frequency: 6382.946808510638 Hz


In [24]:
from mpl_toolkits import mplot3d

%matplotlib qt

lz = 17

fig = plt.figure(figsize =(14, 9))
#ax = plt.axes(projection ='2d')
my_cmap = plt.get_cmap('jet')
x = np.outer(np.linspace(0, 119,118), np.ones(118))
y = x.copy().T
z = np.sqrt(amp_real[lz]**2 + amp_imag[lz]**2)
plt.title(f"Frequency: {parameters[k][f'lz{lz}']['center']} Hz")
plt.imshow( z , cmap = 'jet'  )
plt.scatter(clamp_x,clamp_y,color='WHITE')

### Real data

In [2]:
import hdf5storage
data = hdf5storage.loadmat('data/final_data.mat')
data.keys()

dict_keys(['XX', 'Xmatrix', 'YY', 'ZZ', 'Zfreq', 'f0', 'name2Save', 'sampleintervalS'])

In [3]:
data.keys()

dict_keys(['XX', 'Xmatrix', 'YY', 'ZZ', 'Zfreq', 'f0', 'name2Save', 'sampleintervalS'])

In [26]:
freq_1 = {}
freq_2 = {}
freq_3 = {}
freq_4 = {}
Fc_low = 13300
Fc_high = 3000
v = data['ZZ'][:,0,0][100:]
L = len(v)
Fs = 199999


for i in range(118):
    for j in range(118):
        
        v = data['ZZ'][:,i,j][100:]
        sos_l = signal.bessel(5,Fc_low, 'low', fs=Fs,output='sos') # LPF filtering parameters
        sos_h = signal.bessel(5,Fc_high, 'high', fs=Fs,output='sos') # HPF filtering parameters

        x = signal.sosfilt(sos_l,v) # LPF
        x = signal.sosfilt(sos_h,x) # HPF
        
        freqs=np.fft.fftfreq(L,1/Fs)
        fft= np.fft.fft(x)[freqs>100]
        
        f1 = 5425
        f2 = 7659
        f3 = 6382
        f4 = 12765
        index_freq1 = np.where(freqs>f1)[0][0]
        index_freq2 = np.where(freqs>f2)[0][0]
        index_freq3 = np.where(freqs>f3)[0][0]
        index_freq4 = np.where(freqs>f4)[0][0]
        
        freq_1[f'[{i},{j}]'] = fft[index_freq1]
        freq_2[f'[{i},{j}]'] = fft[index_freq2]
        freq_3[f'[{i},{j}]'] = fft[index_freq3]
        freq_4[f'[{i},{j}]'] = fft[index_freq4]
        

In [35]:
amp_data = np.zeros([118,118])

for i,k in enumerate(freq_4):
    X = int(k.split(',')[0].split('[')[1])
    Y = int(k.split(',')[1].split(']')[0])
    amp_data[X,Y] = np.abs(freq_4[k])



In [36]:
%matplotlib qt


fig = plt.figure(figsize =(14, 9))

#z = np.sqrt(amp_real[lz]**2 + amp_imag[lz]**2)
z = amp_data
plt.title(f"Frequency: 12765 Hz")
plt.imshow( z , cmap = 'jet'  )


plt.scatter(clamp_x,clamp_y,color='white')

###  Clamping

In [42]:
max(clamp_x)

187

In [8]:

file_data = np.genfromtxt("data/clamps.txt", skip_header=0, dtype=int)

clamp_x = (file_data[:,0]+17)/2 
clamp_y = (file_data[:,1]+17)/2

print(clamp_y[0])
clamp_data = np.zeros([118,118])

for i in range(len(clamp_x)):
    #print(clamp_x[i],clamp_y[i])
    x = int(clamp_x[i])
    y = int(clamp_y[i])
    clamp_data[x,y] = 100000


76.0


In [110]:
# gaussian kernel
s = [[1, 2, 1],  
     [0, 0, 0], 
     [-1, -2, -1]]


# convulution
H = signal.convolve2d(z, s)
V = signal.convolve2d(z, np.transpose(s))
R = (H**2 + V**2)**0.5


#cgrafica dicha convolución
plt.figure(figsize = (5, 5))
plt.imshow(R, cmap = 'jet' )